In [1]:
import os
import pandas as pd
import pyspark
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession 
from pyspark.ml.fpm import FPGrowth
import pyspark.sql.functions as f

# Step 1: Define Spark Configuration
conf = SparkConf() \
    .setAppName("PodProcessing") \
    .setMaster("local[4]") \
    .set("spark.executor.memory", "300g") \
    .set("spark.driver.memory", "300g")

# Step 2: Initialize SparkContext with the Configuration
sc = SparkContext(conf=conf)

# Step 3: Initialize SparkSession
spark = SparkSession(sc)

# Now you can continue with your Spark operations


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/local/lib/python3.11/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/lib/python3.11/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/home/jovyan/work/Python-3.11.0/myenv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/jovyan/work/Python-3.11.0/myenv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  F

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/local/lib/python3.11/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/lib/python3.11/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/home/jovyan/work/Python-3.11.0/myenv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/jovyan/work/Python-3.11.0/myenv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  F

AttributeError: _ARRAY_API not found

In [2]:
!pip3.11 install pymarc

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip3.11 install "numpy<2"

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip3.11 install poetry

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip3.11 install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip3.11 install marctable

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import os

# Add the directory to PATH
os.environ['PATH'] += os.pathsep + '/home/jovyan/.local/bin'

# Initial load only

In [14]:
import os
import glob
import tempfile
import logging
from pymarc import MARCReader, Record

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def get_files():
    # Get a list of all MARC files
    files = glob.glob('/home/jovyan/work/columbia-2022-06-15-full-marc21.mrc', recursive=True)
    logger.info(f"Found {len(files)} marc files")
    return files

def process_file(file):
    logger.info(f"Processing file {file}")

    # Define the output directory for Parquet files
    output_dir = '/home/jovyan/work/marc/parquet'
    os.makedirs(output_dir, exist_ok=True)

    # Create a temporary file to store valid MARC records
    try:
        with tempfile.NamedTemporaryFile(delete=False) as temp:
            temp_file = temp.name
    except Exception as e:
        logger.error(f"Error creating temporary file for {file}: {e}")
        return False

    # Process the file in chunks
    try:
        with open(file, 'rb') as f_in, open(temp_file, 'wb') as temp_out:
            if file.endswith('.xml'):
                reader = XMLReader(f_in)
            else:
                reader = MARCReader(f_in)
            for record in reader:
                if not isinstance(record, Record):
                    raise ValueError("Invalid MARC record")
                temp_out.write(record.as_marc())
    except Exception as e:
        logger.error(f"Error processing file {file}: {e}")
        os.remove(temp_file)
        return False

    # Run the marctable command
    output_file = os.path.join(output_dir, os.path.basename(file).replace('.mrc', '.parquet'))
    logger.info(f"Running marctable command: marctable parquet {temp_file} {output_file}")
    exit_status = os.system(f'marctable parquet {temp_file} {output_file}')
    if exit_status != 0:
        logger.error(f"Error executing marctable command for file {file}")
        os.remove(temp_file)
        return False
    else:
        logger.info(f"Created Parquet file {output_file}")

    # Delete the temporary file
    os.remove(temp_file)

    return True

def marc2parquet():
    files = get_files()
    results = []

    for file in files:
        # Check if the corresponding Parquet file already exists
        output_file = os.path.join('/home/jovyan/work/marc/parquet', os.path.basename(file).replace('.mrc', '.parquet'))
        if os.path.exists(output_file):
            logger.info(f"Skipping already processed file {file}")
            continue

        result = process_file(file)
        results.append(result)

    successful_files = sum(results)
    logger.info(f"Processed {len(results)} files, {successful_files} successful, {len(results) - successful_files} failed")

    # Return True if all files were processed successfully, otherwise False
    return all(results)

# Run the function and capture the result
result = marc2parquet()
logger.info(f"marc2parquet result: {result}")

2025-01-31 22:07:01,032 - INFO - Found 1 marc files
2025-01-31 22:07:01,033 - INFO - Processing file /home/jovyan/work/columbia-2022-06-15-full-marc21.mrc
2025-01-31 22:07:01,033 - INFO - Processing file /home/jovyan/work/columbia-2022-06-15-full-marc21.mrc
/home/jovyan/work/Python-3.11.0/myenv/lib/python3.11/site-packages/pymarc/record.py:392: BadSubfieldCodeWarning: 
  warnings.warn(BadSubfieldCodeWarning())
2025-01-31 22:21:56,146 - ERROR - Error processing file /home/jovyan/work/columbia-2022-06-15-full-marc21.mrc: Invalid MARC record
2025-01-31 22:21:57,080 - INFO - Processed 1 files, 0 successful, 1 failed
2025-01-31 22:21:57,081 - INFO - marc2parquet result: False


# Load Parquet

In [15]:
spark_df_penn = spark.read.parquet('/home/jovyan/work/marc/parquet/penn-2022-07-20-full-marc21.parquet')
spark_df_brown = spark.read.parquet('/home/jovyan/work/marc/parquet/brown-2022-06-14-full-marc21.parquet')
spark_df_chicago = spark.read.parquet('/home/jovyan/work/marc/parquet/chicago-2022-06-22-full-marc21.parquet')
spark_df_cornell = spark.read.parquet('/home/jovyan/work/marc/parquet/chicago-2022-06-22-full-marc21.parquet')
# columbia todo
spark_df_dartmouth = spark.read.parquet('/home/jovyan/work/marc/parquet/dartmouth-2022-08-19-full-marc21.parquet')
# duke todo
# harvard todo
spark_df_jhu = spark.read.parquet('/home/jovyan/work/marc/parquet/jhu-2023-08-23-full-marc21.parquet')
spark_df_mit = spark.read.parquet('/home/jovyan/work/marc/parquet/mit-marc21.parquet')
spark_df_princeton = spark.read.parquet('/home/jovyan/work/marc/parquet/princeton-2022-06-17-full-marc21.parquet')
spark_df_stanford = spark.read.parquet('/home/jovyan/work/marc/parquet/stanford-2024-08-28-full-marc21.parquet')
spark_df_yale = spark.read.parquet('/home/jovyan/work/marc/parquet/yale-2022-06-17-full-marc21.parquet')


# Check for unique records across libraries

In [16]:
# using spark sql, I need to query a field 
# and compare the two dataframes to find out how many records are common between 
# the two dataframes and how many are unique to each dataframe
spark_df_penn.createOrReplaceTempView("penn")
spark_df_brown.createOrReplaceTempView("brown")
spark_df_chicago.createOrReplaceTempView("chicago")
spark_df_cornell.createOrReplaceTempView("cornell")
spark_df_dartmouth.createOrReplaceTempView("dartmouth")
spark_df_jhu.createOrReplaceTempView("jhu")
spark_df_mit.createOrReplaceTempView("mit")
spark_df_princeton.createOrReplaceTempView("princeton")
spark_df_stanford.createOrReplaceTempView("stanford")
spark_df_yale.createOrReplaceTempView("yale")

# Query the F245 field to get the OCLC number
penn_oclc = spark.sql("SELECT F245 FROM penn")
brown_oclc = spark.sql("SELECT F245 FROM brown")
chicago_oclc = spark.sql("SELECT F245 FROM chicago")
cornell_oclc = spark.sql("SELECT F245 FROM cornell")
dartmouth_oclc = spark.sql("SELECT F245 FROM dartmouth")
jhu_oclc = spark.sql("SELECT F245 FROM jhu")
mit_oclc = spark.sql("SELECT F245 FROM mit")
princeton_oclc = spark.sql("SELECT F245 FROM princeton")
stanford_oclc = spark.sql("SELECT F245 FROM stanford")
yale_oclc = spark.sql("SELECT F245 FROM yale")

# Find the number of common records in each dataframe
common_oclc = penn_oclc.intersect(brown_oclc).intersect(chicago_oclc).intersect(cornell_oclc).intersect(dartmouth_oclc).intersect(jhu_oclc).intersect(mit_oclc).intersect(princeton_oclc).intersect(stanford_oclc).intersect(yale_oclc)

# Find the number of unique records in each dataframe
unique_penn = penn_oclc.subtract(common_oclc)
unique_brown = brown_oclc.subtract(common_oclc)
unique_chicago = chicago_oclc.subtract(common_oclc)
unique_cornell = cornell_oclc.subtract(common_oclc)
unique_dartmouth = dartmouth_oclc.subtract(common_oclc)
unique_jhu = jhu_oclc.subtract(common_oclc)
unique_mit = mit_oclc.subtract(common_oclc)
unique_princeton = princeton_oclc.subtract(common_oclc)
unique_stanford = stanford_oclc.subtract(common_oclc)
unique_yale = yale_oclc.subtract(common_oclc)

# Print the results
print(f"Number of common records: {common_oclc.count()}")
print(f"Number of unique records in Penn: {unique_penn.count()}")
print(f"Number of unique records in Brown: {unique_brown.count()}")
print(f"Number of unique records in Chicago: {unique_chicago.count()}")
print(f"Number of unique records in Cornell: {unique_cornell.count()}")
print(f"Number of unique records in Dartmouth: {unique_dartmouth.count()}")
print(f"Number of unique records in JHU: {unique_jhu.count()}")
print(f"Number of unique records in MIT: {unique_mit.count()}")
print(f"Number of unique records in Princeton: {unique_princeton.count()}")
print(f"Number of unique records in Stanford: {unique_stanford.count()}")
print(f"Number of unique records in Yale: {unique_yale.count()}")



Number of common records: 59698
Number of unique records in Penn: 6351281
Number of unique records in Brown: 2393694
Number of unique records in Chicago: 7292271
Number of unique records in Cornell: 7292271
Number of unique records in Dartmouth: 1584697
Number of unique records in JHU: 1635214
Number of unique records in MIT: 1338848
Number of unique records in Princeton: 5874929
Number of unique records in Stanford: 8076046
Number of unique records in Yale: 7437992


# Some trial SQL queries (older)

In [ ]:

spark_df.createOrReplaceTempView("df")
df_001 = spark.sql("SELECT `001` FROM df")
# are there any null values?
df_001.filter(df_001['001'].isNull()).count()
#check if all records have 001 values


In [ ]:
#approximate number of items that are portuguese language
por = spark.sql("SELECT * FROM df WHERE `008` LIKE '%por%'")
count_por = por.count()
print(count_por)
